<h1><center>Edmunds - Analytics & Data Science Case Study</center></h1>
<h2><center>Cash for Clunker Analysis</center></h2>
<h3><center>Senior Data Analyst Application</center></h3>
<h3><center>Francis Chen</center></h3>

In [1]:
from IPython.display import HTML, display

display(HTML(data="""
<style>
    div#notebook-container    { width: 63%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Part 0. Data Cleaning
- Raw original data did not contain information on vehicle class
    - Vehicle category was not not sufficiently granular
    - Required joining data from https://www.fueleconomy.gov/feg/ws/index.shtml
- CARS_PAID_FINAL data was joined by:
    - Parsing trade_in_vehicle_drive_train and new_vehicle_drive_train to obtain engine cylinders and displacement information
    - Vehicle class data from fueleconomy website was then joined to CARS_PAID_FINAL data on make, model, cylinders, and displacement

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime as dt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
pd.options.display.float_format = '{:,.4f}'.format
import difflib

In [ ]:
# Data Cleaning and Joining on external database for Vehicle class information

data = pd.read_csv('CARS_PAID_Final.txt', encoding='latin-1',
                   parse_dates = ['invoice_date','sale_date','trade_in_registration_start','trade_in_registration_end'],
                   infer_datetime_format = True,
                   dtype = {'ZIP': str})

# https://www.fueleconomy.gov/feg/ws/index.shtml
vehicles = pd.read_csv('vehicles - original.csv')[['cylinders','displ', 'drive', 'fuelType',
                                                   'make', 'model', 'trany', 'VClass']].drop_duplicates().reset_index(drop = True)
vehicles.model = vehicles.model.str.lower()

def to_float(x):
    try:
        return float(x)
    except:
        return np.nan
    
def to_adate(year):
    return dt.datetime(year,1,1)

# Creating index key to map to vclass for each trade in and new vehicle
trade_ins = data[['trade_in_make',
                  'trade_in_model',
                  'trade_in_vehicle_drive_train']].drop_duplicates().sort_values(['trade_in_make','trade_in_model']).reset_index(drop = True)
trade_ins = trade_ins.join(trade_ins['trade_in_vehicle_drive_train'].str.split(",", expand = True))
trade_ins.columns = ['make','model','descr','displ','cylinders','trany','drive','eng_dscr','fuelType','other']
trade_ins.displ = trade_ins['displ'].str.replace('L',"").apply(lambda x: to_float(x))
trade_ins.cylinders = trade_ins['cylinders'].str.replace(' cyl',"").apply(lambda x: to_float(x))
trade_ins.model = trade_ins.model.str.lower()

new_vehicles = data[['new_vehicle_make',
                     'new_vehicle_model',
                     'new_vehicle_drive_train']].drop_duplicates().sort_values(['new_vehicle_make','new_vehicle_model']).reset_index(drop = True)
new_vehicles = new_vehicles.join(new_vehicles['new_vehicle_drive_train'].str.split(",", expand = True))
new_vehicles.columns = ['make','model','descr','displ','cylinders','trany','drive','eng_dscr','fuelType','other']
new_vehicles.displ = new_vehicles['displ'].str.replace('L',"").apply(lambda x: to_float(x))
new_vehicles.cylinders = new_vehicles['cylinders'].str.replace(' cyl',"").apply(lambda x: to_float(x))
new_vehicles.model = new_vehicles.model.str.lower()

data_keys = pd.concat([trade_ins, new_vehicles], axis = 0).drop_duplicates().reset_index(drop = True)
data_keys = data_keys.merge(vehicles[['make', 'model', 'cylinders','displ','VClass']].drop_duplicates(), how = 'left',
                            on = ['make', 'model', 'cylinders','displ'])

# Cleaning data that did not join properly
data_keys_correct = data_keys[(data_keys.model == 'unlisted') |
                              (data_keys.VClass.isnull() == False)].reset_index(drop = True)
data_keys_incorrect = data_keys[(data_keys.model != 'unlisted') &
                                (data_keys.VClass.isnull())].reset_index(drop = True)

for idx, row in data_keys_incorrect.iterrows():
    temp_vehicles = vehicles[(vehicles.make == row.make)]
    
    if len(temp_vehicles) == 0 and 'sterling' in row.model:
        temp_vehicles = vehicles[(vehicles.make.str.lower() == 'sterling')]
        data_keys_incorrect.loc[idx, 'VClass'] = temp_vehicles[['VClass']].drop_duplicates().reset_index(drop = True).iloc[0,0]
    elif 'Austin Rover' != row.make:
        quick_check = temp_vehicles[['make','model','VClass']].drop_duplicates().reset_index(drop = True)
        temp_join = quick_check.merge(pd.DataFrame(row).T[['make','model']], on = ['make','model'])
        
        if len(temp_join) == 1:
            temp_join = quick_check.merge(pd.DataFrame(row).T[['make','model']], on = ['make','model'])
            data_keys_incorrect.loc[idx, 'VClass'] = temp_join.iloc[0, 2]
        elif row.model == 'apv 2wd':
            data_keys_incorrect.loc[idx, 'VClass'] = 'Van'
            data_keys_incorrect.loc[idx, 'make'] = 'Suzuki'
        else:
            fuzzy_match = vehicles[vehicles.model == difflib.get_close_matches(row.model, vehicles['model'].drop_duplicates())[0]]
            fuzzy_match = fuzzy_match[['make','model','VClass']].drop_duplicates().reset_index(drop = True)
            
            data_keys_incorrect.loc[idx, 'VClass'] = fuzzy_match.loc[0, 'VClass']
            data_keys_incorrect.loc[idx, 'make'] = fuzzy_match.loc[0, 'make']

# Final keys for joining data
class_keys = pd.concat([data_keys_correct, data_keys_incorrect], axis = 0).reset_index(drop = True)
class_keys.make = class_keys.make.str.lower()

trade_in_keys = class_keys.rename(columns = {'make': 'trade_in_make',
                                   'model': 'trade_in_model',
                                   'cylinders': 'trade_in_cylinders',
                                   'trany': 'trade_in_transmission',
                                   'displ': 'trade_in_displacement',
                                   'drive': 'trade_in_drive',
                                   'descr': 'trade_in_vehicle_drive_train'})

new_vehi_keys = class_keys.rename(columns = {'make': 'new_vehicle_make',
                                   'model': 'new_vehicle_model',
                                   'cylinders': 'new_cylinders',
                                   'trany': 'new_transmission',
                                   'displ': 'new_displacement',
                                   'drive': 'new_drive',
                                   'descr': 'new_vehicle_drive_train'})

data.trade_in_make = data.trade_in_make.str.lower()
data.new_vehicle_make = data.new_vehicle_make.str.lower()
data.trade_in_model = data.trade_in_model.str.lower()
data.new_vehicle_model = data.new_vehicle_model.str.lower()

# Mapping initial data to vclass and also additional information
trade_in_join = data[['trade_in_make',
                      'trade_in_model',
                      'trade_in_vehicle_drive_train']].merge(trade_in_keys.drop_duplicates(['trade_in_make',
                                                                            'trade_in_model',
                                                                            'trade_in_vehicle_drive_train']), 
                                             how = 'left',
                                             on = ['trade_in_make',
                                                   'trade_in_model',
                                                   'trade_in_vehicle_drive_train']).rename(columns = {'VClass':'trade_in_vclass'})

new_vehicle_join = data[['new_vehicle_make',
                      'new_vehicle_model',
                      'new_vehicle_drive_train']].merge(new_vehi_keys.drop_duplicates(['new_vehicle_make',
                                                                            'new_vehicle_model',
                                                                            'new_vehicle_drive_train']), 
                                             how = 'left',
                                             on = ['new_vehicle_make',
                                                   'new_vehicle_model',
                                                   'new_vehicle_drive_train']).rename(columns = {'VClass':'new_vehicle_vclass'})


new_data = data.join(trade_in_join[['trade_in_cylinders',
                                    'trade_in_transmission',
                                    'trade_in_displacement',
                                    'trade_in_drive',
                                    'trade_in_vclass']], how = 'left')

new_data = new_data.join(new_vehicle_join[['new_cylinders',
                                    'new_transmission',
                                    'new_displacement',
                                    'new_drive',
                                    'new_vehicle_vclass']], how = 'left')

# Filtering only relevant columns
new_data = new_data[['vendor_id', 'dealer_name', 'address_line1',
                     'city', 'state', 'ZIP', 'invoice_id', 
                     'invoice_date', 'sale_date',
                     'disposal_status', 'disposal_facility_nmvtis_id',
                     'disposal_facility_contact_info', 'sales_type', 'invoice_amount',
                     'trade_in_vehicle_category', 'trade_in_make',
                     'trade_in_model', 'trade_in_year', 'trade_in_vehicle_drive_train',
                     'trade_in_mileage', 'trade_in_title_state',
                     'trade_in_registration_state', 'trade_in_registration_start',
                     'trade_in_registration_end',
                     'trade_in_NMVTIS_flag', 'trade_in_odometer_reading',
                     'new_vehicle_category', 'new_vehicle_make',
                     'new_vehicle_model', 'new_vehicle_year', 'new_vehicle_drive_train',
                     'new_vehicle_car_mileage', 'new_vehicle_MSRP', 'trade_in_vclass',
                     'new_vehicle_vclass','trade_in_displacement', 'trade_in_cylinders', 
                     'trade_in_transmission','trade_in_drive','new_displacement',
                     'new_cylinders','new_transmission', 'new_drive']]

new_data.trade_in_year = new_data.trade_in_year.apply(lambda x: to_adate(x))
new_data['trade_in_years_driven'] = (new_data[['invoice_date','sale_date']].max(axis = 1) - new_data.trade_in_year).dt.days/365.25
new_data['avg_annual_miles_driven'] = new_data.trade_in_odometer_reading/new_data.trade_in_years_driven
new_data['trade_in_annual_fuel_gallons'] = new_data.avg_annual_miles_driven/new_data.trade_in_mileage
new_data['new_vehicle_annual_fuel_gallons'] = new_data.avg_annual_miles_driven/new_data.new_vehicle_car_mileage
# new_data.to_csv('cars_data.csv')

## Part 1. Success Metrics

- Two KPIs were created: <br>
    1. Fuel economy improvement from trade-in vehicle to new vehicle in percent and averaged for each State
    2. Expected annual reduction in fuel consumption in percent for each State and calculated from aggregating individual expected annual fuel consumption for trade-in and new vehicles.

In [3]:
%%HTML
<div class='tableauPlaceholder' id='viz1563379794173' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;TX&#47;TXYQ8MRF8&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;TXYQ8MRF8' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;TX&#47;TXYQ8MRF8&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563379794173');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1020px';vizElement.style.height='887px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Note that the different aggregations for the KPIs mean different weighting implications for each individual <br>
- Average fuel economy improvement for each state is equivalent to assuming every car drives the same distance
- Annual expected reduction in fuel consumption is equivalent to assigning different individual weights for every car based on their average annual mileage, i.e., the fuel economies of frequently used cars will have higher weights and vice versa.

For example, this can be noticed at the country-wide level where Average Fuel Economy Improvement was 59.07%. 
- If every car drove the same distance, then Expected Annual Fuel Reduction would be equal to $1-\frac{1}{1.5907}=0.3713$ or 37.13%.
- However, since we see that the country-wide Expected Annual Fuel Reduction is actually 34.46%, or around 3% lower, this implies that below average fuel economy vehicles are driven more.

## Part 2. Behavioral Analysis

In [5]:
new_data = pd.read_csv('cars_data.csv', encoding='latin-1')

cluster_fields = ['state', 'sales_type', 'trade_in_make', 'trade_in_model',
                  'trade_in_mileage', 'trade_in_odometer_reading',
                  'new_vehicle_make', 'new_vehicle_model', 'new_vehicle_car_mileage',
                  'new_vehicle_MSRP', 'trade_in_vclass', 'trade_in_transmission',
                  'trade_in_drive', 'new_displacement', 'new_cylinders', 'new_transmission',
                  'trade_in_cylinders', 'new_drive', 'trade_in_years_driven',
                  'avg_annual_miles_driven', 'trade_in_annual_fuel_gallons',
                  'new_vehicle_annual_fuel_gallons', 'new_vehicle_vclass']

cluster_data = new_data[cluster_fields].drop_duplicates().reset_index(drop = True)

vclass_dict = {'Compact Cars': 'Compact',
               'Minicompact Cars': 'Compact',
               'Subcompact Cars': 'Compact',
               'Large Cars': 'Large',
               'Midsize Cars': 'Midsize',
               'Midsize Station Wagons': 'St. Wagons',
               'Midsize-Large Station Wagons': 'St. Wagons',
               'Small Station Wagons': 'St. Wagons',
               'Minivan - 2WD': 'Minivans',
               'Minivan - 4WD': 'Minivans',
               'Small Pickup Trucks': 'Pickup',
               'Small Pickup Trucks 2WD': 'Pickup',
               'Small Pickup Trucks 4WD': 'Pickup',
               'Standard Pickup Trucks': 'Pickup',
               'Standard Pickup Trucks 2WD': 'Pickup',
               'Standard Pickup Trucks 4WD': 'Pickup',
               'Special Purpose Vehicle 2WD': 'Special Purpose',
               'Special Purpose Vehicle 4WD': 'Special Purpose',
               'Special Purpose Vehicles': 'Special Purpose',
               'Sport Utility Vehicle - 4WD': 'SUV',
               'Sport Utility Vehicle - 2WD': 'SUV',
               'Standard Sport Utility Vehicle 4WD': 'SUV',
               'Vans Passenger': 'Vans',
               'Vans, Cargo Type': 'Vans',
               'Vans, Passenger Type': 'Vans'}

cluster_data = cluster_data.replace({'trade_in_vclass': vclass_dict,
                                     'new_vehicle_vclass': vclass_dict})

cluster_data = cluster_data[(cluster_data.trade_in_vclass != 'Special Purpose') &
                            (cluster_data['trade_in_odometer_reading'] < 500000) &
                            (cluster_data['trade_in_odometer_reading'] != 0)].reset_index(drop = True)

# cluster_data[['state', 'trade_in_make',
#               'trade_in_mileage', 'trade_in_odometer_reading',
#               'trade_in_vclass','trade_in_years_driven',
#               'new_vehicle_make',
#               'new_vehicle_MSRP',
#               'new_vehicle_vclass',
#               'new_vehicle_car_mileage']].dropna().reset_index(drop = True).to_csv('carclusters_test.csv')

On inspection of the distribution of the classes for trade-in vehicles, we can see that almost half are Special Purpose Vehicles.

In [6]:
trade_in_vclass_summary = cluster_data[['trade_in_vclass',
                                        'new_drive']].groupby('trade_in_vclass').count().reset_index().rename(columns = {'new_drive': 'VClass Count',
                                                                                                           'trade_in_vclass': 'Trade-In VClass'}).sort_values('VClass Count', ascending = False)

data = [go.Bar(x = trade_in_vclass_summary['Trade-In VClass'], # assign x as the dataframe column 'x'
               y = trade_in_vclass_summary['VClass Count'])]

layout = go.Layout(autosize=True,
                   width=950,
                   title='Trade-In VClass Distribution',
                   height=500,
                   xaxis = dict(title = 'Trade-In VClass'),
                   yaxis = dict(title = 'Frequency'))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Given the nature of the Cash for Clunker program, the data would have a selection bias of individuals who want to trade in their original low fuel economy vehicles for new higher fuel economy vehicles. Since the individuals with Special Purpose Vehicles would be more likely to exhibit this bias, their data points were removed before conducting further analysis.

**Clustering Data and Determining Cluster Characteristics** <br>
The data can provide insight into what characteristics of trade-in cars align with the different new vehicles purchased in the program.
- A fast ward clustering algorithm was implemented on the subsetted data (exclusion of Special Purpose trade-in classes and only a select few fields)
- Once each data point was assigned a cluster, a dashboard for the cluster summary was created in Tableau

In [7]:
%%HTML
<div class='tableauPlaceholder' id='viz1563418907360' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;X7&#47;X7C46CZ4C&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;X7C46CZ4C' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;X7&#47;X7C46CZ4C&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563418907360');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1020px';vizElement.style.height='887px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>